In [ ]:
from PyFastBDT import FastBDT
import basf2_mva
import modularAnalysis as ma
import ROOT as root
import numpy as np
import pandas as pd
import math
import uproot
import root_pandas as rp
import analysis_variables as a_v
import matplotlib.pyplot as plt
import seaborn as sn
import datetime
from sklearn.model_selection import train_test_split

In [ ]:
BDT_type = 'contReweight'

sig_path = a_v.input_dir_long + '....parq'
bkg_path = a_v.input_dir_long + '....parq'

sig = pd.read_parquet(sig_path, engine='pyarrow').query('...>-0.15 and ...<0.1 and ...>0.5')
bkg = pd.read_parquet(bkg_path, engine='pyarrow').query('...>-0.15 and ...<0.1 and ...>0.5')#.reset_index()

sig = sig.assign(Signal = [1] * len(sig.d0_Mbc))
bkg = bkg.assign(Signal = [0] * len(bkg.d0_Mbc))

In [ ]:
np.random.seed(0)

cont_tot = bkg

if len(sig.d0_Mbc)>len(bkg.d0_Mbc):
    sig = sig.sample(frac=len(bkg.d0_Mbc)/len(sig.d0_Mbc), weights='...')
else:
    bkg = bkg.sample(frac=len(sig.d0_Mbc)/len(bkg.d0_Mbc), weights='...')
    
cont_used = bkg
a = len(cont_used)/len(cont_tot) * 200 
print('Used Continuum:',a,'fb-1')
    
train_percent = 0.8
test_percent = 1 - train_percent

sig_y = sig.Signal
sig_x = sig.drop('Signal',axis=1)
bkg_y = bkg.Signal
bkg_x = bkg.drop('Signal',axis=1)

x = pd.concat([sig_x,bkg_x])
y = pd.concat([sig_y,bkg_y])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_percent)

train = x_train.assign(Signal = y_train)
test = x_test.assign(Signal = y_test)

train.to_parquet(a_v.input_dir +'BDT_' + BDT_type + '_train_' + str(datetime.date.today()) + '.parq', compression='GZIP')
test.to_parquet(a_v.input_dir +'BDT_' + BDT_type + '_test_' + str(datetime.date.today()) + '.parq', compression='GZIP')

In [ ]:
BDT_type = 'contSupp_BB' #contSupp contSupp_BB 

bkg_path = a_v.input_dir + '....parq'

sig = cont_tot.drop(cont_used.index)
bkg = pd.read_parquet(bkg_path, engine='pyarrow').query('...>-0.15 and ...<0.1 and ...>0.5')

sig = sig.assign(Signal = [1] * len(sig.d0_Mbc))
bkg = bkg.assign(Signal = [0] * len(bkg.d0_Mbc))

In [ ]:
if len(sig.d0_Mbc)>len(bkg.d0_Mbc):
    sig = sig.sample(frac=len(bkg.d0_Mbc)/len(sig.d0_Mbc), weights='...')
else:
    bkg = bkg.sample(frac=len(sig.d0_Mbc)/len(bkg.d0_Mbc), weights='...')

cont_used_2 = sig
b = len(cont_used_2)/len(cont_tot) * 200
print('Used Continuum:',b,'fb-1')
    
train_percent = 0.8
test_percent = 1 - train_percent

sig_y = sig.Signal
sig_x = sig.drop('Signal',axis=1)
bkg_y = bkg.Signal
bkg_x = bkg.drop('Signal',axis=1)

x = pd.concat([sig_x,bkg_x])
y = pd.concat([sig_y,bkg_y])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_percent)

train = x_train.assign(Signal = y_train)
test = x_test.assign(Signal = y_test)

train.to_parquet(a_v.input_dir +'BDT_' + BDT_type + '_train_' + str(datetime.date.today()) + '.parq', compression='GZIP')
test.to_parquet(a_v.input_dir +'BDT_' + BDT_type + '_test_' + str(datetime.date.today()) + '.parq', compression='GZIP')